
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Reading from a Streaming Query

## Objectives
1. Build a streaming DataFrame
1. Display streaming query results
1. Write streaming query results to table
1. Monitor the streaming query


### Classes referenced
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.DataStreamReader.html" target="_blank">DataStreamReader</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.DataStreamWriter.html" target="_blank">DataStreamWriter</a>
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.StreamingQuery.html" target="_blank">StreamingQuery</a>

In [0]:
%run ./Includes/Classroom-Setup-01

## Prototyping in Batch Mode

Explore the dataset and test out transformation logic using batch dataframes.

In [0]:
from pyspark.sql.functions import col, approx_count_distinct, count

batch_df = (spark.read  
              .load(DA.paths.events)
              .filter(col("traffic_source") == "email")
              .withColumn("mobile", col("device").isin(["iOS", "Android"]))
              .select("user_id", "event_timestamp", "mobile")
           )

print(batch_df.isStreaming)

display(batch_df)


## Build streaming DataFrames

Switching from batch to stream is easy! 

Change the code from `spark.read` to `spark.readStream` - everything else, including the transformation logic remain unchanged.

In [0]:
from pyspark.sql.functions import col, approx_count_distinct, count
 
streaming_df = (spark.readStream.load(DA.paths.events)
                      .filter(col("traffic_source") == "email")
                      .withColumn("mobile", col("device").isin(["iOS", "Android"]))
                      .select("user_id", "event_timestamp", "mobile")
                   )

print(streaming_df.isStreaming)

display(streaming_df, streamName = "display_user_devices")


### Write streaming query results

Take the final streaming DataFrame (our result table) and write it to a Delta Table sink in `append` mode. In this labs setup, the table will be created in Unity Catalog.

**NOTE:** `append` mode is the default mode when writing stateless queries to sink.

In [0]:
checkpoint_path = f"{DA.paths.working_dir}/email_traffic"

devices_query = (streaming_df
                  .writeStream
                  .outputMode("append")
                  .format("delta")
                  .queryName("email_traffic")
                  .trigger(processingTime="1 second")
                  .option("checkpointLocation", checkpoint_path)
                  .toTable("email_traffic_only")
                )

### Monitor streaming query

Use the streaming query handle to monitor and control it.

In [0]:
devices_query.name


Let's see the query status.

In [0]:
devices_query.status


[lastProgress](https://spark.apache.org/docs/3.5.2/structured-streaming-programming-guide.html#managing-streaming-queries) gives us metrics from the previous query

In [0]:
devices_query.lastProgress

In [0]:
import time
# Run for 10 more seconds
time.sleep(10) 

devices_query.stop()

In [0]:
devices_query.awaitTermination()


[awaitTermination](https://spark.apache.org/docs/3.5.2/structured-streaming-programming-guide.html#managing-streaming-queries) 
 blocks the current thread until the streaming query is terminated.

For stand-alone structured streaming applications, this is used to prevent the main thread from terminating while the streaming query is still executing. In the context of this training environment, it's useful in case you use **Run all** to run the notebook. This prevents subsequent command cells from executing until the streaming query has fully terminated.

In [0]:
# Stop the streaming query with the query's name or query's Id
result = stop_streaming_query("email_traffic")
print(result)


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>